In [1]:
#standard packages
import pandas as pd
import numpy as np

#webscraping packages & SQLite
import pymongo
from pymongo import MongoClient
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import functions as mf

In [2]:
client = MongoClient()
db = client.case_files
cases = db.cases

In [3]:
df = pd.DataFrame(list(cases.find()))

In [4]:
def combine_other_opinions(case_text_entry):
    return [case_text_entry[i]['type'] + ': ' + case_text_entry[i]['text'] for i in range(1, len(case_text_entry))]

In [5]:
def clean_data(df):
    df['attorneys'] = df['attorneys'].apply(lambda x: None if len(x) == 0  else x[0])
    df['judges'] = df['judges'].apply(lambda x: None if len(x) == 0 else x[0])
    df['majority_opinion'] = df['case_text'].apply(lambda x: None if len(x) == 0 else (x[0]['type'] + ': ' + x[0]['text']))
    df['other_opinions'] = df['case_text'].apply(lambda x: combine_other_opinions(x))
    df['other_opinions'] = df['other_opinions'].apply(lambda x: None if len(x) == 0 else x[0])
    return df

In [6]:
test = clean_data(df)
test.head()

,_id,attorneys,case_id,case_name,case_text,court_id,court_name,decision_date,frontend_url,judges,majority_opinion,other_opinions
0,5d66e470e59ddf33befdfe06,None,37588,In the matter of the Complaint made against Jo...,"[{'text': 'ORDER OF PUBLIC CENSURE Whereas, In...",8874,Kansas Supreme Court,1970,https://cite.case.law/kan/206/335/,None,"majority: ORDER OF PUBLIC CENSURE\nWhereas, In...",None
1,5d66e470e59ddf33befdfe07,None,37656,In the matter of the Complaint made against Ro...,"[{'text': 'ORDER OF PUBLIC CENSURE Whereas, In...",8874,Kansas Supreme Court,1970,https://cite.case.law/kan/206/334/,None,"majority: ORDER OF PUBLIC CENSURE\nWhereas, In...",None
2,5d66e470e59ddf33befdfe08,None,1747634,State of Connecticut v. Anonymous (1971-5),"[{'text': 'Moraghan, J. The defendant has move...",13405,Connecticut Circuit Court,1970,https://cite.case.law/conn-cir-ct/6/443/,None,"majority: Moraghan, J.\nThe defendant has move...",None
3,5d66e470e59ddf33befdfe09,None,1747661,State of Connecticut v. Anonymous (1971-3),"[{'text': 'Dearington, J. Following a trial to...",13405,Connecticut Circuit Court,1970,https://cite.case.law/conn-cir-ct/6/393/,None,"majority: Dearington, J.\nFollowing a trial to...",None
4,5d66e470e59ddf33befdfe0a,None,1747680,State of Connecticut v. Anonymous (1971-7),"[{'text': 'Casale, J. The defendant, after a t...",13405,Connecticut Circuit Court,1970,https://cite.case.law/conn-cir-ct/6/462/,None,"majority: Casale, J.\nThe defendant, after a t...",None


In [69]:
for i in range(10, 18):
    print(i, test.attorneys[i])

10 J. Marvin Kuhn, Deputy Public Defender, Salem, filed the brief for appellant. With him on the brief was Gary D. Babcock, Public Defender, Salem.
11 Bertram B. Wallace, pro se.
12 William H. Baker, Special Counsel, Bobby L. Culpepper, Jonesboro, La., for defendants-appellants.
13 Lacy Mahon, Jr., Jacksonville, Fla., for appellant.
14 Paul L. Wayman, Atlanta, Ga., court-appointed, for appellant.
15 Elihu Pearlman for plaintiff.
16 None
17 John J. Harrington for the petitioner.


In [58]:
temp = []
for i in range(0,150):
    if len(df.case_text[i]) > 1: 
        temp.append(i)

In [59]:
temp

[5, 98, 121]

In [40]:
# for i,x in enumerate(db.cases.find({}, {'decision_date':1})):
#     print(i)
#     print(x)